For **GRIM/TFT** *vs.* **All-D** 

(for other cases, we need to simply modify the energy matrix 'E', while the code structure remains the same)

1. **Agent-based modelling** (plot of $\mu^{ABM}$ vs *reward* $\mathbb{R}$)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def magnetization(PU, a, T, beta, w):
    m = []
    T = 5.0; S = 0.0; P = 1.0
    for R in PU:
        E = [[-R/(1-w), -S -w*P/(1-w)], [-T - w*P/(1-w), -P/(1-w)]] 
        for k in range(800000):  
            i = np.random.randint(len(a))  
            p = np.random.rand()  
            ΔE = E[a[i]][a[(i + 1) % len(a)]] - E[(a[i] + 1) % 2][a[(i + 1) % len(a)]]
            if p <= (1 / (1 + np.exp(-beta * ΔE / T))):
                a[i] = (a[i] + 1) % 2  
        m.append((len(a) - 2 * np.sum(a)) / len(a))  
    return m

a = np.random.rand(1000)
PU = np.linspace(1.0, 5.0, 1000)  
for i in range(len(a)):
    if a[i] < 0.5:
        a[i] = 0
    else:
        a[i] = 1

a = a.astype(int)  

beta_values = [1.0, 2.0, 10.0]
w_values = [0.9]

filename = "repeated-mag-var-w.csv"     #saves the data in a .CSV file.
with open(filename, 'w') as file:
    file.write("PU,Beta,w,Magnetization\n")
    for beta in beta_values:
        for w in w_values:
            m_values = magnetization(PU, a.copy(), T=1.0, beta=beta, w=w)
            for i, PU_val in enumerate(PU):
                file.write(f"{PU_val},{beta},{w},{m_values[i]}\n")

plt.figure(figsize=(10, 8))
for beta in beta_values:
    for w in w_values:
        m_values = magnetization(PU, a.copy(), T=1.0, beta = beta, w = w)
        plt.plot(PU, m_values, label=f'β = {beta}, $\mathcal{{w}}$ = {w}', linewidth="2")
plt.xlabel('$\mathbb{R}$', fontsize=35)
plt.ylabel('$\mu^{ABM}$', fontsize=35)
plt.legend(fontsize=15, loc='lower right')
plt.grid()
plt.show()

To replot from the .CSV file

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt("repeated-mag-test.csv", delimiter=',', skiprows=1)

PU_values = np.unique(data[:, 0])
beta_values = np.unique(data[:, 1])
w_values = np.unique(data[:, 2])

plt.figure(figsize=(7, 5))
for beta in beta_values:
    for w in w_values:
        mask = (data[:, 1] == beta) & (data[:, 2] == w)
        m_values = data[mask][:, 3]
        plt.plot(PU_values, m_values, label=f'$\mathbb{{T}} = 5.0$, $\mathbb{{S}} = 0.0$, $\mathbb{{P}} = 1.0$, β = {beta}, $\mathcal{{w}}$ = 0.9', linewidth="1")
plt.xlabel('$\mathbb{R}$', fontsize=35)
plt.ylabel('$\mu^{ABM}$', fontsize=35)
plt.legend(fontsize=12, loc='lower right')
plt.grid()
plt.show()

2. **Agent-based modelling** (plot of $\mu^{ABM}$ vs *discount factor* $w$)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def magnetization(w_values, a, T, beta, R):
    m = []
    T = 5.0; S = 0.0; P = 1.0
    for w in w_values:  
        E = [[-R/(1-w), -S -w*P/(1-w)], [-T - w*P/(1-w), -P/(1-w)]]
        for _ in range(500000):  
            i = np.random.randint(len(a))  
            p = np.random.rand()  
            ΔE = E[a[i]][a[(i + 1) % len(a)]] - E[(a[i] + 1) % 2][a[(i + 1) % len(a)]]
            if p <= (1 / (1 + np.exp(-beta * ΔE / T))):
                a[i] = (a[i] + 1) % 2  
        m.append((len(a) - 2 * np.sum(a)) / len(a))  
    return m

a = np.random.rand(1000)
w_values = np.linspace(0, 0.999, 1000)
for i in range(len(a)):
    if a[i] < 0.5:
        a[i] = 0
    else:
        a[i] = 1

a = a.astype(int)  

R_vals = [3.0]
beta_values = [1.0, 2.0, 3.0]

filename = "repeated-mag-w.csv"     #saves the data in a .CSV file
with open(filename, 'w') as file:
    file.write("w_values,Beta,R,Magnetization\n")
    for beta in beta_values:
        for R in R_vals:
            m_values = magnetization(w_values, a.copy(), T=1.0, beta=beta, R=R)
            for i, w_v in enumerate(w_values):
                file.write(f"{w_v},{beta},{R},{m_values[i]}\n")


plt.figure(figsize=(10, 8))
for beta in beta_values:
    for R in R_vals:
        m_values = magnetization(w_values, a.copy(), T=1.0, beta=beta, R=R)
        plt.plot(w_values, m_values, label=f'β = {beta}', linewidth="2")
plt.xlabel('$\mathcal{w}$', fontsize=35)
plt.ylabel('$\mu^{ABM}$', fontsize=35)
plt.legend(fontsize=15, loc='lower right')
plt.grid()
plt.show()

To replot from the .CSV file

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv

filename = "repeated-mag-w.csv"
w_values = []
beta_values = []
R_values = []
magnetization_values = []

with open(filename, 'r') as file:
    reader = csv.reader(file)
    next(reader) 
    for row in reader:
        w_values.append(float(row[0]))
        beta_values.append(float(row[1]))
        R_values.append(float(row[2]))
        magnetization_values.append(float(row[3]))

num_w = len(set(w_values))
num_betas = len(set(beta_values))
w_values = np.array(w_values).reshape(num_betas, num_w)
magnetization_values = np.array(magnetization_values).reshape(num_betas, num_w)

plt.figure(figsize=(7, 5))
for i, beta in enumerate(set(beta_values)):
    plt.plot(w_values[i], magnetization_values[i], label=f'$\mathbb{{T}} = 5.0$, $\mathbb{{R}} = 3.0$, $\mathbb{{P}} = 1.0$, $\mathbb{{S}} = 0.0$, β = {beta}', linewidth=1)

plt.xlabel('$\mathcal{w}$', fontsize=35)
plt.ylabel('$\mu^{ABM}$', fontsize=35)
plt.legend(fontsize=11, loc='upper left')
plt.grid()
plt.show()